### Задача 2.1

Импортируем необходимые библиотеки и модули

In [1]:
import findspark
findspark.init('/home/joann/spark-3.1.1-bin-hadoop3.2')

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col

Пропишем путь к файлам и создадим spark-сессию

In [3]:
PATH = '/home/joann/src'

spark = SparkSession \
        .builder \
        .appName('olympic_athletes') \
        .enableHiveSupport() \
        .getOrCreate()

spark.sparkContext.setLogLevel('OFF')

**Задание 1**

Сгенерировать DataFrame из трёх колонок (row_id, discipline, season) - олимпийские дисциплины по сезонам.
* row_id - число порядкового номера строки;
* discipline - наименование олимпиский дисциплины на английском (полностью маленькими буквами);
* season - сезон дисциплины (summer / winter);

Укажите не мнее чем по 5 дисциплин для каждого сезона.

Сохраните DataFrame в csv-файл, разделитель колонок табуляция, первая строка должна содержать название колонок.

Данные должны быть сохранены в виде 1 csv-файла, а не множества маленьких.

Сформируем переменные, содержащие данные и схему

В данных десять строк отведем летним дисциплинам (т.к. их в целом больше, чем зимним), и пять строк - зимним

In [4]:
data = [(1, 'badminton', 'summer'),
        (2, 'basketball', 'summer'),
        (3, 'boxing', 'summer'),
        (4, 'golf', 'summer'),
        (5, 'hockey', 'summer'),
        (6, 'karate', 'summer'),
        (7, 'skateboarding', 'summer'),
        (8, 'swimming', 'summer'),
        (9, 'tennis', 'summer'),
        (10, 'volleyball', 'summer'),
        (11, 'biathlon', 'winter'),
        (12, 'bobsleigh', 'winter'),
        (13, 'curling', 'winter'),
        (14, 'luge', 'winter'),
        (15, 'snowboard', 'winter')]

schema = '''row_id bigint,
            discipline string,
            season string'''

Создадим датафрейм и выведем его на экран

In [5]:
df_olympic_sports = spark.createDataFrame(data, schema)

df_olympic_sports.show()

+------+-------------+------+
|row_id|   discipline|season|
+------+-------------+------+
|     1|    badminton|summer|
|     2|   basketball|summer|
|     3|       boxing|summer|
|     4|         golf|summer|
|     5|       hockey|summer|
|     6|       karate|summer|
|     7|skateboarding|summer|
|     8|     swimming|summer|
|     9|       tennis|summer|
|    10|   volleyball|summer|
|    11|     biathlon|winter|
|    12|    bobsleigh|winter|
|    13|      curling|winter|
|    14|         luge|winter|
|    15|    snowboard|winter|
+------+-------------+------+



Сохраним датафрейм в csv файл и прочитаем его, чтобы проверить корректность сохранения

In [6]:
df_olympic_sports\
    .repartition(1)\
    .write\
    .mode('overwrite')\
    .options(header=True, delimiter='\t')\
    .csv(f'{PATH}/olympic_sports.csv')

spark\
    .read\
    .options(header=True, delimiter='\t')\
    .csv(f'{PATH}/olympic_sports.csv')\
    .show()

+------+-------------+------+
|row_id|   discipline|season|
+------+-------------+------+
|     1|    badminton|summer|
|     2|   basketball|summer|
|     3|       boxing|summer|
|     4|         golf|summer|
|     5|       hockey|summer|
|     6|       karate|summer|
|     7|skateboarding|summer|
|     8|     swimming|summer|
|     9|       tennis|summer|
|    10|   volleyball|summer|
|    11|     biathlon|winter|
|    12|    bobsleigh|winter|
|    13|      curling|winter|
|    14|         luge|winter|
|    15|    snowboard|winter|
+------+-------------+------+



**Задание 2**

Прочитайте исходный файл "Athletes.csv".

Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.

Результат сохраните в формате parquet.

Прочитаем файл Athletes.csv и выведем на экран первые десять строк

In [7]:
df_athletes = spark\
                .read\
                .options(header=True, delimiter=';')\
                .csv(f'{PATH}/Athletes.csv')

df_athletes.show(10)

+-----------------+--------------------+-------------------+
|             Name|                 NOC|         Discipline|
+-----------------+--------------------+-------------------+
|  AALERUD Katrine|              Norway|       Cycling Road|
|      ABAD Nestor|               Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|               Italy|             Rowing|
|   ABALDE Alberto|               Spain|         Basketball|
|    ABALDE Tamara|               Spain|         Basketball|
|        ABALO Luc|              France|           Handball|
|     ABAROA Cesar|               Chile|             Rowing|
|    ABASS Abobakr|               Sudan|           Swimming|
| ABBASALI Hamideh|Islamic Republic ...|             Karate|
|    ABBASOV Islam|          Azerbaijan|          Wrestling|
+-----------------+--------------------+-------------------+
only showing top 10 rows



Сгруппируем данные по столбцу Discipline и посчитаем количество значений

In [8]:
df_pivot_athletes = df_athletes\
                        .groupBy('Discipline')\
                        .count()

df_pivot_athletes\
    .sort('Discipline')\
    .show(10)

+-------------------+-----+
|         Discipline|count|
+-------------------+-----+
|     3x3 Basketball|   62|
|            Archery|  122|
|Artistic Gymnastics|  187|
|  Artistic Swimming|   98|
|          Athletics| 2068|
|          Badminton|  164|
|  Baseball/Softball|  220|
|         Basketball|  280|
|   Beach Volleyball|   90|
|             Boxing|  270|
+-------------------+-----+
only showing top 10 rows



Сохраним датафрейм в файл parquet и прочитаем его, чтобы проверить корректность сохранения

In [10]:
df_pivot_athletes\
    .write\
    .mode('overwrite')\
    .options(header=True)\
    .parquet(f'{PATH}/pivot_athletes.parquet')

spark\
    .read\
    .parquet(f'{PATH}/pivot_athletes.parquet')\
    .sort('Discipline')\
    .show(10)

+-------------------+-----+
|         Discipline|count|
+-------------------+-----+
|     3x3 Basketball|   62|
|            Archery|  122|
|Artistic Gymnastics|  187|
|  Artistic Swimming|   98|
|          Athletics| 2068|
|          Badminton|  164|
|  Baseball/Softball|  220|
|         Basketball|  280|
|   Beach Volleyball|   90|
|             Boxing|  270|
+-------------------+-----+
only showing top 10 rows



**Задание 3**

Прочитайте исходный файл "Athletes.csv".

Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.

Получившийся результат нужно объединить с сгенерированным вами DataFrame из 1-го задания и в итоге вывести количество участников только по тем дисциплинам, что есть в вашем сгенерированном DataFrame.

Результат сохраните в формате parquet.

Объединим датафреймы по столбцу с дисциплинами, заполним NaN нулями, удалим лишний столбец с дисциплинами

In [11]:
df_total_pivot = df_olympic_sports\
    .join(df_pivot_athletes, lower(df_pivot_athletes.Discipline) == df_olympic_sports.discipline, 'left')\
    .fillna(value=0, subset='count')\
    .drop(df_pivot_athletes.Discipline)

df_total_pivot.show()

+------+-------------+------+-----+
|row_id|   discipline|season|count|
+------+-------------+------+-----+
|     1|    badminton|summer|  164|
|     2|   basketball|summer|  280|
|     3|       boxing|summer|  270|
|     4|         golf|summer|  115|
|     5|       hockey|summer|  406|
|     6|       karate|summer|   77|
|     7|skateboarding|summer|   77|
|     8|     swimming|summer|  743|
|     9|       tennis|summer|  178|
|    10|   volleyball|summer|  274|
|    11|     biathlon|winter|    0|
|    12|    bobsleigh|winter|    0|
|    13|      curling|winter|    0|
|    14|         luge|winter|    0|
|    15|    snowboard|winter|    0|
+------+-------------+------+-----+



Сохраним датафрейм в файл parquet и прочитаем его, чтобы проверить корректность сохранения

In [12]:
df_total_pivot\
    .write\
    .mode('overwrite')\
    .options(header=True)\
    .parquet(f'{PATH}/total_pivot.parquet')

spark\
    .read\
    .parquet(f'{PATH}/total_pivot.parquet')\
    .sort('row_id')\
    .show()

+------+-------------+------+-----+
|row_id|   discipline|season|count|
+------+-------------+------+-----+
|     1|    badminton|summer|  164|
|     2|   basketball|summer|  280|
|     3|       boxing|summer|  270|
|     4|         golf|summer|  115|
|     5|       hockey|summer|  406|
|     6|       karate|summer|   77|
|     7|skateboarding|summer|   77|
|     8|     swimming|summer|  743|
|     9|       tennis|summer|  178|
|    10|   volleyball|summer|  274|
|    11|     biathlon|winter|    0|
|    12|    bobsleigh|winter|    0|
|    13|      curling|winter|    0|
|    14|         luge|winter|    0|
|    15|    snowboard|winter|    0|
+------+-------------+------+-----+

